<a href="https://colab.research.google.com/github/GrzegorzSzczepanek/deep_learning_fun/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [487]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report, roc_curve, roc_auc_score
import torch
from torch import nn

In [488]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [489]:
dataset = pd.read_csv("/content/drive/MyDrive/neuron_workplace/autentykacja_eeg.csv", delimiter=";")
dataset.head()

,Delta,Theta,Alpha_low,Alpha_high,Beta_low,Beta_high,Gamma_low,Gamma_high,Attention,Meditation,BlinkStrength,Flag
0,785389,114375,57106,37625,24464,17104,15846,8024,45,91,15,1
1,386949,80792,23506,22118,9566,7147,5392,1697,8,42,6,0
2,21453,14085,4756,5062,2176,2117,2867,862,48,66,9,1
3,322637,56936,17810,25793,15870,8050,3621,2100,25,69,8,0
4,19412,12985,6416,6131,7126,7419,2677,1884,80,53,-1,1


In [490]:
# In next try do a train, validation and test splits

X = torch.tensor(dataset.loc[:, "Delta": "BlinkStrength"].values, dtype=torch.float32)
y = torch.tensor(dataset.Flag.values, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [491]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([4195, 11]),
 torch.Size([1049, 11]),
 torch.Size([4195]),
 torch.Size([1049]))

In [492]:
y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)

# 1. Najpierw spróbuję napisać MLP na wszystkich danych bez dokonywania w nich zmian.
W moim datasecie nie ma pustych krotek. Na razie pominę standaryzację, by porównać efekty potem

In [493]:
X, y

(tensor([[ 7.8539e+05,  1.1438e+05,  5.7106e+04,  ...,  4.5000e+01,
           9.1000e+01,  1.5000e+01],
         [ 3.8695e+05,  8.0792e+04,  2.3506e+04,  ...,  8.0000e+00,
           4.2000e+01,  6.0000e+00],
         [ 2.1453e+04,  1.4085e+04,  4.7560e+03,  ...,  4.8000e+01,
           6.6000e+01,  9.0000e+00],
         ...,
         [ 8.0033e+04,  3.2374e+04,  7.0020e+03,  ...,  6.1000e+01,
           5.3000e+01,  1.5000e+01],
         [ 1.6945e+05,  3.8482e+04,  1.2956e+04,  ...,  6.2000e+01,
           8.7000e+01,  1.6000e+01],
         [ 1.0141e+05,  4.1010e+04,  1.0225e+04,  ...,  4.8000e+01,
           3.7000e+01, -1.0000e+00]]),
 tensor([1., 0., 1.,  ..., 0., 0., 1.]))

In [494]:
input = len(dataset.columns.values) - 1
# output = len(y.unique())


class FlagModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU()
        )

        self.block_2 = nn.Sequential(
            nn.Linear(hidden_units, 1),
            nn.Sigmoid()  # Sigmoid activation for binary classification
        )


    def forward(self, x):
        return self.block_2(self.block_1(x))


flag_model_1 = FlagModel(input, 10, 1).to(device)
flag_model_1

FlagModel(
  (block_1): Sequential(
    (0): Linear(in_features=11, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): ReLU()
  )
  (block_2): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [495]:
def accuracy_fn(y_true: torch.Tensor, y_pred: torch.Tensor) -> [torch.float]:
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.BCELoss()

optimizer = torch.optim.Adam(params=flag_model_1.parameters(), lr=0.01)

EPOCHS = 1001

In [496]:
def train_step(model: torch.nn.Module,
               data: torch.tensor,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    train_loss, train_acc = 0, 0
    model.train()
    # print(data.shape)
    # print(y_train.shape)

    logits = model(data)
    pred = torch.round(torch.sigmoid(logits))

    loss = loss_fn(logits, y_train)
    acc = accuracy_fn(y_true=y_train,
                      y_pred=pred)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    return loss, acc



In [497]:
def test_step(model: torch.nn.Module,
               data: torch.tensor,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    model.eval()
    with torch.inference_mode():
        logits = model(data)
        pred = torch.round(torch.sigmoid(logits))
        loss = loss_fn(logits, y_test)
        acc = accuracy_fn(y_true=y_test, y_pred=pred)

        return loss, acc



In [498]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

train_time_start = timer()

torch.manual_seed(42)

def train(model: torch.nn.Module,
          train_data: torch.tensor,
          test_data: torch.tensor,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          epochs: int,
          device:torch.device = device):

    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = train_step(model=model,
                                        data=train_data,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        accuracy_fn=accuracy_fn)
        test_loss, test_acc = test_step(model=model,
                                        data=test_data,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        accuracy_fn=accuracy_fn)

        if epoch % 100 == 0:
            print(f"Epoch: {epoch} | Train Loss: {train_loss} | Train Acc: {train_acc:.0f}% | Test loss: {test_loss} | Test Acc: {test_acc:.0f}%")

    train_time_end = timer()
    print(f"Training was done in {(train_time_end - train_time_start):.2f} seconds")

In [499]:
train(model=flag_model_1,
      train_data=X_train,
      test_data=X_test,
      loss_fn=loss_fn,
      optimizer=optimizer,
      accuracy_fn=accuracy_fn,
      epochs=EPOCHS)

  0%|          | 0/1001 [00:00<?, ?it/s]

Epoch: 0 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 100 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 200 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 300 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 400 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 500 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 600 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 700 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 800 | Train Loss: 59.6662712097168 | Train Acc: 40% | Test loss: 61.868446350097656 | Test Acc: 38%
Epoch: 900 | Train Loss: 59.66627120971